#### 1. scala使用中括号表示泛型  
#### 2. scala函数也可以指定泛型  
#### 3. 可以指定泛型T的上界, 表示T是某个类的子类
1. 使用符号`<:`  
2. 如下例子, scala的String类实现了Comparable接口, 因此泛型T可以调用`compareTo`接口  
 但调用`new Pair(2,1)`就会失败, 因为scala的Int没有实现`Comparable`接口. 解决办法在第5点

In [9]:
class Pair[T <: Comparable[T]](first:T,second:T){
    def smaller = {
        if (first.compareTo(second) <0 ) first else second
    }
}
new Pair("2","1").smaller

defined class Pair
res8_1: String = "1"

#### 4. 指定泛型下界, 表示泛型是下界类型的父类
使用符号`>:`

In [4]:
class Pair[T](first:T,second:T){
    //使用T的父类型替换第一个参数
    def replaceFirst[R>:T](newFirst:R) = new Pair[R](newFirst,second)
}

defined class Pair

#### 5. 视图界定
1. 如第3点所示, scala的Int没有实现`Comparable`接口, 但scala的RichInt实现了`Comparable`接口, 且scala包含Int到RichInt的隐式转换.  
2. scala的视图界定, 符号`<%`, 意味泛型T可以通过某种隐式转换后, 变成其上界的子类

In [10]:
class Pair[T <% Comparable[T]](first:T,second:T){
    def smaller = {
        if (first.compareTo(second) <0 ) first else second
    }
}
print(new Pair(2,1).smaller)

1

defined class Pair

#### 6. 上下文界定
1. 泛型`T:M`, 表示上下文中存在一个类型为`M[T]`的隐式值.  
2. 当需要使用这个隐式值时, 通过方法的隐士参数调用这个隐式值

In [8]:
class Pair[T:Ordering](first:T,second:T){
    def smaller(implicit ord:Ordering[T]) = {
        if(ord.compare(first,second)<0) first else second
    }
}
print(new Pair("2","1").smaller)

1

defined class Pair

#### 7. Manifest记录数组型泛型
1. 因为scala的泛型信息在编译时会被擦除,所以当我们想要构造泛型数组时,类型Array[T]在运行时并不知道T是什么.  
 因此应使用Menifest类保存泛型信息,并将这个信息创建成隐士变量.而scala编译器, 会自动为如下2种Manifest类型创建隐士变量:  
   1. 利用上下文界定,告知编译器创建存储泛型信息的隐士变量: `[T:Manifest]`  
   2. 利用函数的隐士参数, 告知编译器创建存储泛型信息的隐士变量: `(implicit m:Manifest[T])`, 变量名m可变

In [14]:
class Pair[T:Manifest](first:T,second:T){
    def makePair() = {
        val r = new Array[T](2)
        r(0) = first
        r(1) = second
        r
    }
}
val arr = new Pair(1,2).makePair
arr.foreach(x=>print(x+","))

1,2,

defined class Pair
arr: Array[Int] = Array(1, 2)

2. 作为隐士参数的Menifest举例

In [15]:
// def arr[T] = new Array[T](0)                          // does not compile, 运行时不知道T是什么
def arr[T](implicit m: Manifest[T]) = new Array[T](0) // compiles
def arr2[T: Manifest] = new Array[T](0)                // shorthand for the preceding

defined function arr
defined function arr2

3. Manifest的问题  
 Manifest是scala2.8推出的, 它虽然能保存泛型信息,  但当泛型是scala的内部类时, 却无法区分不同对象的内部类, 如下例子.

In [8]:
class Outer{
    class Inner{}
}

val out1 = new Outer
val inner1 = new out1.Inner()  // 内部类实例
val out2 = new Outer
val inner2 = new out2.Inner()  // 内部类实例

// 柯里化后, 第二个才能使用第一个参数
def getEnv(out:Outer)(inner:out.Inner)(implicit en:Manifest[out.Inner]) = en

// 返回True, 这个是错误的, 因为scala的内部类从属于对象, 因此out1.Inner不等于out2.Inner, 应返回False
println(getEnv(out1)(inner1) == getEnv(out2)(inner2))

true


defined class Outer
out1: wrapper.wrapper.Outer = $sess.cmd7Wrapper$Helper$Outer@27ccbd7b
inner1: wrapper.wrapper.out1.Inner = $sess.cmd7Wrapper$Helper$Outer$Inner@33f74ef5
out2: wrapper.wrapper.Outer = $sess.cmd7Wrapper$Helper$Outer@6e1e184d
inner2: wrapper.wrapper.out2.Inner = $sess.cmd7Wrapper$Helper$Outer$Inner@1f10e3db
defined function getEnv

4. TypeTag  
  为解决内部类路径问题, scala2.10推出TypeTag来修正Manifest的问题.

In [9]:
import scala.reflect.runtime.universe._
class Outer{
    class Inner{}
}

val out1 = new Outer
val inner1 = new out1.Inner()  // 内部类实例
val out2 = new Outer
val inner2 = new out2.Inner()  // 内部类实例

// 柯里化后, 第二个才能使用第一个参数
def getEnv(out:Outer)(inner:out.Inner)(implicit en:TypeTag[out.Inner]) = en

// 返回False,表明问题已修正
println(getEnv(out1)(inner1) == getEnv(out2)(inner2))

false


import scala.reflect.runtime.universe._

defined class Outer
out1: wrapper.wrapper.Outer = $sess.cmd8Wrapper$Helper$Outer@48ed934e
inner1: wrapper.wrapper.out1.Inner = $sess.cmd8Wrapper$Helper$Outer$Inner@74900dcd
out2: wrapper.wrapper.Outer = $sess.cmd8Wrapper$Helper$Outer@34dc05f7
inner2: wrapper.wrapper.out2.Inner = $sess.cmd8Wrapper$Helper$Outer$Inner@49a388e9
defined function getEnv

5. ClassTag  
  1. TypeTag可以保存任意深度嵌套的泛型信息, 如List[Set[String]], 而ClassTag只能保存到List[String]  
  2. 通常ClassTag已可以解决泛型擦除问题, 而且不存在内部类异常问题  
  3. ClassTag与TypeTag是同时推出的

#### 8. 多重界定
如下`T<: Int: Manifest`表示: T是Int的子类, 且存在Menifest[T]的隐式值记录泛型

In [22]:
def arr2[T<: Int: Manifest]() = new Array[T](1)  

defined function arr2

In [23]:
val a = arr2[Int]
a(0) = 1
a.foreach(print)

1

a: Array[Int] = Array(1)

#### 9. 泛型测试
1. 符号`<:<`,`>:>`,`<%<`  
2. 使用场景:  
  有时类上的泛型是宽泛的, 但类中的某些方法又要求该泛型满足某些条件才能执行, 这是就可以在方法上定义泛型测试

In [29]:
class Pair[T](first:T,second:T){
    def smaller(implicit ev: T<:<Ordered[T]) = {
        if(first<second) first else second
    }
}

// 如上定义,虽然可以实例化对象类型为Pair[File],但该对象在调用smaller时会报错, 因为FIle无法通过类型测试

defined class Pair

3. 另一个应用场景:  
 泛型测试的另一个应用场景是: 有时方法有2个泛型, 而一个泛型又是.另一个泛型的泛型, 此时比哪一期无法同时推断出这2个泛型的关系.  
 此时的解决办法是, 通过柯里化, 分步推断出这2个泛型

In [30]:
// 该定义泛型A是C的泛型, 参数it只依赖泛型C, 因此编译器无法推断出泛型A, 接下来的调用会报错
def firstLast[A,C<:Iterable[A]](it:C) = (it.head,it.last)
// 报错:推断出的类型[Nothing,List[Int]],不满足泛型绑定[A,C <: Iterable[A]]
firstLast(List(1,2,3))

cmd30.sc:3: inferred type arguments [Nothing,List[Int]] do not conform to method firstLast's type parameter bounds [A,C <: Iterable[A]]
val res30_1 = firstLast(List(1,2,3))
              ^cmd30.sc:3: type mismatch;
 found   : List[Int]
 required: C
val res30_1 = firstLast(List(1,2,3))
                            ^

: 

In [33]:
// 柯里化解决方法
def firstLast[A,C](it:C)(implicit ev: C<:<Iterable[A]) = (it.head,it.last)
// 编译器先推断出泛型C,在推断出泛型A
firstLast(List(1,2,3))

defined function firstLast
res32_1: (Int, Int) = (1, 3)

#### 10. 泛型协变
如下声明: `Pair[+T]`, 表示若Student类是People类的子类, 则Pair[Student]是Pair[People]的子类

In [36]:
class People{}
class Student extends People{}
class Pair[+T]{}

val s = new Student()
println(s.isInstanceOf[People])

val pair = new Pair[Student]
println(pair.isInstanceOf[Pair[People]]) //证明Pair[Student]是Pair[People]的子类

true
true


defined class People
defined class Student
defined class Pair
s: wrapper.wrapper.Student = $sess.cmd35Wrapper$Helper$Student@5eb4ba2c
pair: wrapper.wrapper.Pair[wrapper.wrapper.Student] = $sess.cmd35Wrapper$Helper$Pair@2556e4cd